In [112]:
import pandas as pd

In [113]:
df_drugs = pd.read_csv('Data/ucr_drug_arrests_2019_cleaned.csv')
df_crime = pd.read_csv('Data/ucr_arrests_2019_cleaned.csv')
df_other = pd.read_csv('Data/ucr_other_arrests_2019_cleaned.csv')

In [114]:
#len of each df
print(len(df_drugs))
print(len(df_crime))
print(len(df_other))

19452
19452
19452


In [115]:
df_crime_grouped = df_crime.groupby(['fips_state_county_code']).agg({"population":"sum","agg_assault_tot_arrests":"sum","arson_tot_arrests":"sum","burglary_tot_arrests":"sum","mtr_veh_theft_tot_arrests":"sum","murder_tot_arrests":"sum","rape_tot_arrests":"sum","robbery_tot_arrests":"sum","theft_tot_arrests":"sum"
})
df_drugs_grouped = df_drugs.groupby(['fips_state_county_code']).agg({"sale_cannabis_tot_arrests":"sum","sale_heroin_coke_tot_arrests":"sum","sale_other_drug_tot_arrests":"sum","sale_synth_narc_tot_arrests":"sum","poss_cannabis_tot_arrests":"sum","poss_heroin_coke_tot_arrests":"sum","poss_other_drug_tot_arrests":"sum","poss_synth_narc_tot_arrests":"sum",
})
df_other_crime_grouped = df_other.groupby(['fips_state_county_code']).agg({"all_other_tot_arrests":"sum","curfew_loiter_tot_arrests":"sum","disorder_cond_tot_arrests":"sum","family_off_tot_arrests":"sum","manslaught_neg_tot_arrests":"sum","oth_assault_tot_arrests":"sum","oth_sex_off_tot_arrests":"sum","prostitution_tot_arrests":"sum","runaways_tot_arrests":"sum","suspicion_tot_arrests":"sum","vagrancy_tot_arrests":"sum","vandalism_tot_arrests":"sum","weapons_tot_arrests":"sum",
})

In [116]:
#len of grouped dfs
print(len(df_crime_grouped))
print(len(df_drugs_grouped))
print(len(df_other_crime_grouped))

3138
3138
3138


In [117]:
df_all_crime = df_crime_grouped.merge(df_drugs_grouped, on='fips_state_county_code', how='inner')
df_all_crime = df_all_crime.merge(df_other_crime_grouped, on='fips_state_county_code', how='inner')


In [118]:
print(len(df_all_crime))

3138


In [119]:
df_all_crime.to_csv('Data_Cleaned/all_crime_arrests_2019_cleaned.csv')

In [120]:
#add all df_all_crime columns together to make a crime_total column
df_all_crime['total_crime'] = df_all_crime.sum(axis=1) - df_all_crime['population']
print(len(df_all_crime))

3138


In [121]:
df_all_crime_total = df_all_crime[['population', 'total_crime']]
df_all_crime_total['crime_per_10k'] = df_all_crime_total['total_crime']/df_all_crime_total['population']*10000
df_all_crime_total = df_all_crime_total['crime_per_10k']
print(len(df_all_crime))

3138


C:\Users\steve\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [122]:
df_all_crime_total.head()
df_all_crime_total.to_csv('Data_Cleaned/model_crime_data.csv')
print(len(df_all_crime))

3138


In [123]:
######### SEGMENTING CRIME TYPES #########
#reset the index
df_all_crime_total = df_all_crime_total.reset_index(inplace=False)
df_all_crime = df_all_crime.reset_index(inplace=False)

In [124]:
print(len(df_all_crime_total))
print(len(df_all_crime))

3138
3138


In [125]:
df_all_crime_total["violent_crime"] = (df_all_crime['agg_assault_tot_arrests'] + df_all_crime['arson_tot_arrests'] + df_all_crime['burglary_tot_arrests'] + df_all_crime['mtr_veh_theft_tot_arrests'] + df_all_crime['murder_tot_arrests'] + df_all_crime[ 'rape_tot_arrests'] + df_all_crime['robbery_tot_arrests'] + df_all_crime['oth_assault_tot_arrests']) / df_all_crime["population"] *10000
df_all_crime_total["hard_drugs_crime"] = (df_all_crime['sale_heroin_coke_tot_arrests'] + df_all_crime['sale_synth_narc_tot_arrests'] + df_all_crime['poss_heroin_coke_tot_arrests'] + df_all_crime['poss_synth_narc_tot_arrests']) / df_all_crime["population"] *10000
df_all_crime_total["soft_drugs_crime"] = (df_all_crime['sale_cannabis_tot_arrests'] + df_all_crime['sale_other_drug_tot_arrests'] + df_all_crime['poss_cannabis_tot_arrests'] + df_all_crime['poss_other_drug_tot_arrests']) / df_all_crime["population"] *10000
df_all_crime_total['commerce_crimes'] = (df_all_crime['burglary_tot_arrests'] + df_all_crime['mtr_veh_theft_tot_arrests'] + df_all_crime['theft_tot_arrests'] + df_all_crime['sale_heroin_coke_tot_arrests'] + df_all_crime['sale_other_drug_tot_arrests'] + df_all_crime['sale_synth_narc_tot_arrests'] + df_all_crime['prostitution_tot_arrests']) / df_all_crime["population"] *10000

In [126]:
df_all_crime_total = df_all_crime_total.rename(columns={'violent_crime':'violent_crime_per_10k', 'hard_drugs_crime':'hard_drugs_crime_per_10k', 'soft_drugs_crime':'soft_drugs_crime_per_10k', 'commerce_crimes':'commerce_crimes_per_10k', "fips_state_county_code":"FIPS"})
df_all_crime_total.to_csv('Data_Cleaned/model_crime_data_segmented.csv')